In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from jax.experimental import stax
import matplotlib.pyplot as plt
from dm_control import suite
import numpy as np
import jax

In [ ]:
'''
class actor_critic
- mlp (shared params for now)
- forward
- getactionprobs?

class PPO
- memory(states, actions, rewards, action_probs, is_terminal)
- actor_critic
- loss(params, memories)
- update(params)
    for n_updates:
        for minibatch in memories: (importance sampling?)
            grads = dloss(params, memories)
            update model params
        clear memory

train(env, max_episodes, batch_size, minibatch_size, 
      K, eps_clip, gamma, lr, lambda, gamma)
      
      make env
      make ppo
      while epochs<epochs:
          get exp, add to memory
          if nmemories==batch_size:
              update
              epochs += 1
          
'''

In [7]:
# dm_control wrapper
class Env():
    def __init__(self, domain, task):
        self.env = suite.load(domain, task)
        time_step = self.env.reset()
        self.action_dim = self.env.action_spec().shape[0]
        self.state_dim = len(self.process_time_step(time_step)[0])
    
    def step(self, action):
        timestep = self.env.step(action)
        return self.process_time_step(timestep)
        
    def reset(self):
        return self.process_time_step(self.env.reset())
        
    @staticmethod
    def process_time_step(time_step):
        """ Convert 'dm_env._environment.TimeStep' to (state, reward, done) """
        state = np.hstack([v for v in time_step.observation.values()])
        return state, time_step.reward, time_step.last()

In [11]:
env = Env('fish', 'upright')
env.step([0,0,0,0,0])
env.state_dim

21

In [11]:
def mlp(key, input_dim, output_dim, hidden_layers=(64,32)):
    layers = []
    
    for hl in hidden_layers:
        layers += [stax.Dense(hl), stax.Tanh()]
    layers.append(stax.Dense(output_dim))
    
    init_fun, apply_fun = stax.serial(*layers)
    params = init_fun(key, -1, input_dim)
    
    return apply_fun, params




        


        

In [14]:
class ActorCritic():
    
    def __init__(state_dim, action_dim, seed=0,
                 actor_hlayers=(64,32), critic_hlayers=(64,32)):
        
        key  = jax.random.PRNGKey(seed)
        keys = jax.random.split(key, num=2)
        
        self.actor, actor_params   = mlp(keys[0], state_dim, action_dim, actor_hlayers)
        self.critic, critic_params = mlp(keys[1], state_dim, 1, critic_hlayers)
        self.params = {'actor': actor_params, 'critic': critic_params}
    
    def act(state: np.array, std):
#         return action, action_prob
        pass
    
    def evaluate(state: np.array, action: np.array, std):
#         return action_prob, value, entropy
        pass

# tests